# Modeling for Ozone Variability Project

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
import numpy as np
import os

## Additional Pre-Processing

In [ ]:
data = pd.read_csv('./final_data/alldata.csv')

In [ ]:
data['Arithmetic Mean'].shape

In [ ]:
data['date'] = pd.to_datetime(data['Date Local'])

In [ ]:
data = data.sort_values(by=['Site Num', 'Date Local'])

In [ ]:
columns_to_check = [
    'Arithmetic Mean', 
    'Arithmetic Mean_press', 
    'Arithmetic Mean_temp', 
    'Arithmetic Mean_rhdp', 
    'Arithmetic Mean_winddirection', 
    'Arithmetic Mean_windspeed'
]

# Count the number of valid (non-NA) values for each of the specified columns
valid_counts = data[columns_to_check].count()

valid_counts


In [ ]:
columns_to_check = ['Arithmetic Mean', 'Arithmetic Mean_temp',  
                    'Arithmetic Mean_rhdp', 'Arithmetic Mean_winddirection', 'Arithmetic Mean_windspeed']

data = data.dropna(subset=columns_to_check)

In [ ]:
data.to_csv('./final_data/nopress_andna.csv', index=False)

In [ ]:
data.shape

## Modeling with all values

In [ ]:
data = pd.read_csv('./final_data/nopress_andna.csv')

In [ ]:
row_counts = data['Local Site Name'].value_counts()
row_counts


In [ ]:
columns_to_preserve = [
    'Date Local',
    'Arithmetic Mean',  
    'Arithmetic Mean_temp', 
    'Arithmetic Mean_rhdp', 
    'Arithmetic Mean_winddirection', 
    'Arithmetic Mean_windspeed'
]

min_data = data[columns_to_preserve]

In [ ]:
min_data.to_csv('./final_data/new_min_data.csv', index=False)

In [ ]:
min_data.hist(bins=50, figsize=(15, 10))

# Display the histograms
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Define features and target
features = [
    'Arithmetic Mean_temp', 
    'Arithmetic Mean_rhdp', 
    'Arithmetic Mean_winddirection', 
    'Arithmetic Mean_windspeed'
]

X = min_data[features]
y = min_data['Arithmetic Mean']

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training (70%), development (10%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42)

# Define the parameter grid for Random Forest
rf_param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Create a Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

# Random search of parameters for Random Forest
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=rf_param_dist, 
                               n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the Random Forest model
rf_random.fit(X_train, y_train)

# Evaluate the best Random Forest model on the development set
best_rf = rf_random.best_estimator_
rf_dev_pred = best_rf.predict(X_dev)
rf_dev_rmse = mean_squared_error(y_dev, rf_dev_pred, squared=False)
rf_dev_r2 = r2_score(y_dev, rf_dev_pred)

# Define the parameter grid for SVM
svm_param_dist = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# Create a Support Vector Machine Regressor
svm = SVR()

# Random search of parameters for SVM
svm_random = RandomizedSearchCV(estimator=svm, param_distributions=svm_param_dist, 
                                n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the SVM model
svm_random.fit(X_train, y_train)

# Evaluate the best SVM model on the development set
best_svm = svm_random.best_estimator_
svm_dev_pred = best_svm.predict(X_dev)
svm_dev_rmse = mean_squared_error(y_dev, svm_dev_pred, squared=False)
svm_dev_r2 = r2_score(y_dev, svm_dev_pred)

# Print only the metrics typically necessary for a research paper for the best models
print("Random Forest - Development RMSE:", rf_dev_rmse, "Development R^2:", rf_dev_r2)
print("Support Vector Machine - Development RMSE:", svm_dev_rmse, "Development R^2:", svm_dev_r2)


In [ ]:
print(best_rf)
print(best_svm)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Initialize Linear Regression and Ridge Regression models
lr_model = LinearRegression()
ridge_model = Ridge()

# Train Linear Regression model
lr_model.fit(X_train, y_train)

# Make predictions on the development set
lr_dev_pred = lr_model.predict(X_dev)

# Calculate RMSE and R^2 for Linear Regression on the development set
lr_dev_rmse = mean_squared_error(y_dev, lr_dev_pred, squared=False)
lr_dev_r2 = r2_score(y_dev, lr_dev_pred)

# Train Ridge Regression model
ridge_model.fit(X_train, y_train)

# Make predictions on the development set
ridge_dev_pred = ridge_model.predict(X_dev)

# Calculate RMSE and R^2 for Ridge Regression on the development set
ridge_dev_rmse = mean_squared_error(y_dev, ridge_dev_pred, squared=False)
ridge_dev_r2 = r2_score(y_dev, ridge_dev_pred)

# Print the evaluation metrics for Linear Regression
print("Linear Regression - Development RMSE:", lr_dev_rmse, "Development R^2:", lr_dev_r2)

# Print the evaluation metrics for Ridge Regression
print("Ridge Regression - Development RMSE:", ridge_dev_rmse, "Development R^2:", ridge_dev_r2)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Define the Random Forest model with the specified parameters
rf_model = RandomForestRegressor(
    max_depth=10,
    max_features='log2',
    min_samples_leaf=2,
    n_estimators=500,
    random_state=42
)

# Train the Random Forest model on the training set
rf_model.fit(X_train, y_train)

# Make predictions on the training and test sets
rf_train_pred = rf_model.predict(X_train)
rf_test_pred = rf_model.predict(X_test)

# Calculate RMSE and R^2 for the Random Forest model on the training set
rf_train_rmse = mean_squared_error(y_train, rf_train_pred, squared=False)
rf_train_r2 = r2_score(y_train, rf_train_pred)

# Calculate RMSE and R^2 for the Random Forest model on the test set
rf_test_rmse = mean_squared_error(y_test, rf_test_pred, squared=False)
rf_test_r2 = r2_score(y_test, rf_test_pred)

# Print the evaluation metrics for the Random Forest model
print("Random Forest - Training RMSE:", rf_train_rmse)
print("Random Forest - Training R^2:", rf_train_r2)
print("Random Forest - Test RMSE:", rf_test_rmse)
print("Random Forest - Test R^2:", rf_test_r2)

In [ ]:
# Scatter plot with fitted line for training and test data (for Linear Regression)
plt.figure(figsize=(10, 6))
plt.scatter(y_train, rf_train_pred, color='blue', label='Training Data')
plt.scatter(y_test, rf_test_pred, color='red', label='Test Data')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted (Random Forest)')
plt.legend()
plt.show()


In [ ]:
# Get feature importances from the Random Forest model
feature_importance = rf_model.feature_importances_

# Create a DataFrame to display feature names and their importance values
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})

# Sort the DataFrame by importance values in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print("Feature Importances:")
print(feature_importance_df)